# Refined NMF for Our AMT Project  
$X_{r,n,m} = {\Sigma}^K_{k=1}G_{r,k} {\cdot} A_{n,k} {\cdot} S_{m,k}$  

 - 随机初始化$B,G$  
     
     - $B$迭代：${B = B.{\times} {\frac{{\frac{X}{BG}}G^T}{1G^T}}}$  
     - $G$迭代：${G = G.{\times} {\frac{{\nabla}{c^-}(B,G)}{{\nabla}{c^+}(B,G)}}}$，其中：${{\nabla}{c^+}(B,G)} = {B^T}1$，${{\nabla}{c^-}(B,G)} = {B^T{\frac{X}{BG}}}$
     - Cost function: ${c(B,G)=c_r(B,G)+{\alpha}c_t(G)+{\beta}c_s(G)}$  
        - $c_r(B,G)$: Reconstruction error term, using: 
	     ${D(X||BG)={\Sigma}_{k,t}[X]_{k,t}{\log}{\frac{[X]_{k,t}}{[BG]_{k,t}}}-[X]_{k,t} + [BG]_{k,t}}$  
		  
		  $c_t(G)$: Temporal continuity term, using: 
		  ${c_t(G)={\Sigma}_{j=1}^{J}{\frac{1}{{\sigma}_j^2}}{\Sigma}_{t=2}^T(g_{t,j}-g_{t-1,j})^2}, {\sigma}_j={\sqrt{(1/T){\Sigma}_{t=1}^{T}g_{t,j}^2}}$  
		- $c_s(G)$: Sparseness Objective term, using: 
		  $c_s(G) = {\Sigma}_{j=1}^J{\Sigma}_{t=1}^Tf(g_{j,t}/{\sigma}_j)$  

In [1]:
import numpy as np
from librosa.core import icqt
import librosa.display
import IPython.display
import librosa, librosa.util

In [2]:
A = np.array([[1,2,3], [3,2,1]])
B = np.array([[1,4], [2,5], [3,6]])
C = np.array([[2,3], [4,7]])

A = np.mat(A)
B = np.mat(B)
C = np.mat(C)

${B = B.{\times} {\frac{{\frac{X}{BG}}G^T}{1G^T}}}$  

In [15]:
def update_B(X, B, G):
    # B, X, G are all np.matrix
    numerator = X / (B * G)
    numerator = numerator * G.T

    one = np.ones(X.shape)
    denominator = one * G.T
    
    factor = numerator / denominator
    
    return  np.multiply(B, factor)

${G = G.{\times} {\frac{{\nabla}{c^-}(B,G)}{{\nabla}{c^+}(B,G)}}}$  
${{\nabla}{c^+}(B,G)} = {B^T}1$  
${{\nabla}{c^-}(B,G)} = {B^T{\frac{X}{BG}}}$

In [4]:
def update_G(X, B, G):
    one = np.ones(X.shape)
    nabla_p = B.T * one

    nabla_n = B.T * (X / (B * G))
    
    factor = nabla_n / nabla_p
    
    return np.multiply(G, factor)

$c_r(B,G)$: Reconstruction error term, using:   
${D(X||BG)={\Sigma}_{k,t}([X]_{k,t}{\log}{\frac{[X]_{k,t}}{[BG]_{k,t}}}-[X]_{k,t} + [BG]_{k,t})}$  

In [34]:
def get_reconstruction_error(X, B, G):
    X_re =  B * G
    log_value = np.log(np.divide(X, X_re))
    multi_value = np.multiply(X, log_value)
    sub_value = np.subtract(multi_value, X)
    add_value = np.add(sub_value, X_re)
    
    return np.sum(add_value)

${\sigma}_j={\sqrt{(1/T){\Sigma}_{t=1}^{T}g_{t,j}^2}}$  

In [33]:
def get_deviation(G):
    # square deviation of each component
    T = G.shape[1]
    square = np.multiply(G, G)
    j_sum = np.sum(square, axis=1)
    
    return j_sum/T

$c_t(G)$: Temporal continuity term, using:  
${c_t(G)={\Sigma}_{j=1}^{J}{\frac{1}{{\sigma}_j^2}}{\Sigma}_{t=2}^T(g_{t,j}-g_{t-1,j})^2}$

In [32]:
def get_temporal_term(G, deviation):
    G_diff = np.subtract(G[:, 1:], G[:, 0:-1])
    square = np.multiply(G_diff, G_diff)
    G_sum = np.sum(square, axis=1)
    
    time_sum = np.divide(G_sum, deviation)
    j_sum = np.sum(time_sum)
    
    return j_sum

$c_s(G)$: Sparseness Objective term, using:   
$c_s(G) = {\Sigma}_{j=1}^J{\Sigma}_{t=1}^Tf(g_{j,t}/{\sigma}_j)$  

In [8]:
def get_sparseness_term(G, deviation):
    std = np.sqrt(deviation)
    sparse_matrix = np.abs(G / std)
    
    return np.sum(sparse_matrix)

In [31]:
def get_cost(X, B, G, alpha, beta):
    # reconstruction error
    c_r = get_reconstruction_error(X, B, G)
    sqr_deviation = get_deviation(G)
    c_t = get_temporal_term(G, sqr_deviation)
    c_s = get_sparseness_term(G, sqr_deviation)
    
    return c_r + alpha * c_t+ beta * c_s

In [52]:
def nmf(C, n, max_iterate, alpha, beta, tolerance=0.08):
    X = np.matrix(C)
    
    # initialize B and G randomly
    B = np.matrix(np.random.rand(X.shape[0], n))
    G = np.matrix(np.random.rand(n, X.shape[1]))
    
    cost = []
    for i in range(0, max_iterate):
        B = update_B(X, B, G)
        G = update_G(X, B, G)
        
        if i == 0:
            # cost[1] - previous cost, cost[0] - current cost
            cost.append(get_cost(X, B, G, alpha, beta))
            cost.append(cost[0]+1)
        else:
            cost[1] = cost[0]
            cost[0] = get_cost(X, B, G, alpha, beta)
            
        if np.abs(cost[0]-cost[1]) <= tolerance:
            break
            
    return B, G, cost[0]

In [60]:
y ,sr = librosa.load('./resource/violin.wav')
C = librosa.cqt(y, sr=sr, hop_length=64, n_bins=84, bins_per_octave=12)
C_n = np.abs(C)

S_full, phase = librosa.magphase(C)
print(type(S_full[0][0]), type(phase[0][0]))

<class 'numpy.float64'> <class 'numpy.complex128'>


In [61]:
B, G, cost = nmf(S_full, 2, 200, 0.5, 0.5)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in multiply
  after removing the cwd from sys.path.


In [54]:
print(B.shape, G.shape, cost)

(84, 2) (2, 1593) (nan+nanj)


In [66]:
def synthesis(B, G, phase, sr):
    W = np.array(B)
    H = np.array(G)
    
    n_W = []
    n_H = []
    n_S = []
    n_C = []
    n_wav = []
    
    n_components = W.shape[1]
    for i in range(0, n_components):
        w = W[:,i:i+1]
        h = H[i:i+1, :]
        s = np.multiply(w, h)
        c = s * phase
        wav = icqt(c, sr=sr, hop_length=64)

        n_W.append(w)
        n_H.append(h)
        n_S.append(s)
        n_C.append(c)
        n_wav.append(wav)
        
    return n_wav

In [67]:
n_wav = synthesis(B, G, phase, sr)

/usr/local/lib/python3.5/dist-packages/librosa/core/constantq.py:615: UserWarning: librosa.icqt is unstable, and subject to change in future versions. Please use with caution.
  warnings.warn('librosa.icqt is unstable, and subject to change in future versions. '
/usr/local/lib/python3.5/dist-packages/librosa/core/constantq.py:615: UserWarning: librosa.icqt is unstable, and subject to change in future versions. Please use with caution.
  warnings.warn('librosa.icqt is unstable, and subject to change in future versions. '


In [68]:
IPython.display.Audio(data=n_wav[0], rate=sr)

In [70]:
IPython.display.Audio(data=n_wav[1], rate=sr)